In [ ]:
# Install necessary packages (run once)
!pip install transformers datasets peft accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# 1. Load pre-trained model & tokenizer
model_name = "decapoda-research/llama-7b-hf"  # Replace with your base model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],  # typical target modules for LoRA in LLaMA
    inference_mode=False,
)
model = get_peft_model(model, lora_config)

# 3. Prepare a small customer support dataset
# Example data - replace with your real data
data = [
    {
        "instruction": "Customer: My package arrived damaged. Help me.",
        "response": "I’m sorry to hear that. We’ll arrange a replacement immediately."
    },
    {
        "instruction": "Customer: I want to return my order.",
        "response": "Sure, I can help you with the return process. Please provide your order number."
    },
    {
        "instruction": "Customer: When will my order arrive?",
        "response": "Let me check the status for you. Could you please share your order ID?"
    },
]

# 4. Create a Dataset and tokenize it
def format_example(example):
    # Combine instruction and response as input-output pair for causal LM
    full_text = f"{example['instruction']}\nResponse: {example['response']}"
    return {"text": full_text}

dataset = Dataset.from_list(data)
dataset = dataset.map(format_example)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "response", "text"])
tokenized_dataset.set_format("torch")

# 5. Define training arguments
training_args = TrainingArguments(
    output_dir="./customer_support_lora",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    evaluation_strategy="no",
    learning_rate=2e-4,
    fp16=True,  # mixed precision if supported
)

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 7. Train the model
trainer.train()

# 8. Save the fine-tuned model
model.save_pretrained("./customer_support_lora")
tokenizer.save_pretrained("./customer_support_lora")
